# Mounting Drive and installing missing librairies

In [6]:
%%capture

from google.colab import drive
drive.mount('/content/gdrive')

!pip install pytorch_lightning
!pip install ipywidgets
!pip install rich
!pip install captum

# Notebook settings

In [7]:
do_training = True

# Imports

In [8]:
from zipfile import ZipFile

from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader
import torchvision.transforms as transforms

import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision.models import ResNet50_Weights
# Voir si c'est torch ou torchvision à ce niveau
from torchvision.utils import _log_api_usage_once
from torchvision.models._api import WeightsEnum, Weights
from torchvision.models._meta import _IMAGENET_CATEGORIES
from torchvision.models._utils import handle_legacy_interface, _ovewrite_named_param

import pytorch_lightning as pl

from typing import Any, Dict, Callable, Tuple, Optional, Type, Union, List

from pytorch_lightning.callbacks import RichProgressBar, ModelCheckpoint
from pytorch_lightning.callbacks.progress.rich_progress import RichProgressBarTheme

# from captum.attr import DeepLift, DeepLiftShap, GradientShap, Saliency
import captum.attr as explain

# Torchvision model redefining

In [9]:
def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation,
    )


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError("BasicBlock only supports groups=1 and base_width=64")
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU()
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion: int = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.0)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU()
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        num_classes: int = 1000,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        _log_api_usage_once(self)
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                f"or a 3-element tuple, got {replace_stride_with_dilation}"
            )
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck) and m.bn3.weight is not None:
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock) and m.bn2.weight is not None:
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        planes: int,
        blocks: int,
        stride: int = 1,
        dilate: bool = False,
    ) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes, planes, stride, downsample, self.groups, self.base_width, previous_dilation, norm_layer
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                )
            )

        return nn.Sequential(*layers)

    def _forward_impl(self, x: torch.Tensor) -> torch.Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self._forward_impl(x)


def _resnet(
    block: Type[Union[BasicBlock, Bottleneck]],
    layers: List[int],
    weights: Optional[WeightsEnum],
    progress: bool,
    **kwargs: Any,
) -> ResNet:
    if weights is not None:
        _ovewrite_named_param(kwargs, "num_classes", len(weights.meta["categories"]))

    model = ResNet(block, layers, **kwargs)

    if weights is not None:
        model.load_state_dict(weights.get_state_dict(progress=progress))

    return model


_COMMON_META = {
    "min_size": (1, 1),
    "categories": _IMAGENET_CATEGORIES,
}

def resnet50(*, weights: Optional[ResNet50_Weights] = None, progress: bool = True, **kwargs: Any) -> ResNet:
    """ResNet-50 from `Deep Residual Learning for Image Recognition <https://arxiv.org/pdf/1512.03385.pdf>`__.

    .. note::
       The bottleneck of TorchVision places the stride for downsampling to the second 3x3
       convolution while the original paper places it to the first 1x1 convolution.
       This variant improves the accuracy and is known as `ResNet V1.5
       <https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch>`_.

    Args:
        weights (:class:`~torchvision.models.ResNet50_Weights`, optional): The
            pretrained weights to use. See
            :class:`~torchvision.models.ResNet50_Weights` below for
            more details, and possible values. By default, no pre-trained
            weights are used.
        progress (bool, optional): If True, displays a progress bar of the
            download to stderr. Default is True.
        **kwargs: parameters passed to the ``torchvision.models.resnet.ResNet``
            base class. Please refer to the `source code
            <https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py>`_
            for more details about this class.

    .. autoclass:: torchvision.models.ResNet50_Weights
        :members:
    """
    weights = ResNet50_Weights.verify(weights)

    return _resnet(Bottleneck, [3, 4, 6, 3], weights, progress, **kwargs)

# Extract and load data

## Extract and load train/test sets

In [10]:
archive_path = '/content/gdrive/MyDrive/datasets/fruits_vegetables_360.zip'

with ZipFile(archive_path, mode='r') as zip:
  zip.extractall('/content/fruits_vegetables_360/')

train_set = ImageFolder('/content/fruits_vegetables_360/fruits-360_dataset/fruits-360/Training/', transform=transforms.ToTensor())
test_set = ImageFolder('/content/fruits_vegetables_360/fruits-360_dataset/fruits-360/Test/', transform=transforms.ToTensor())

## Split training set into training and validation sets

In [11]:
train_ratio = 0.9
total_size = len(train_set)

train_size = int(train_ratio * total_size)
valid_size = total_size - train_size

train_set, valid_set = random_split(train_set, [train_size, valid_size])

Look at set sizes

In [12]:
len(train_set), len(valid_set), len(test_set)

(60922, 6770, 22688)

# ResNet
Simple to use and simple to use explainability process on it.

## Load and freeze weights for finetuning

We freeze the entire model weights and biases just to keep classifier learnable.  
We want to keep initial features extractor.

In [13]:
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)

model.fc = nn.Linear(in_features=2048, out_features=131)

for name, params in model.named_parameters():
  if 'fc' in name:
    continue
  params.requires_grad = False

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

# PyTorch Lightning wrapper

In [14]:
class LightningWrapper(pl.LightningModule):
  def __init__(self, model: nn.Module, loss_function: Callable=F.cross_entropy, optimizer: torch.optim.Optimizer=torch.optim.Adam, 
               optimizer_params: Dict[str, Any]={'lr': 0.001}, **pl_module) -> None:
    super(LightningWrapper, self).__init__(**pl_module)
    self.save_hyperparameters()
    self.wrapped_model = model
    self.loss_function = loss_function
    self.optimizer = optimizer
    self.optimizer_params = optimizer_params

  def configure_optimizers(self) -> torch.optim.Optimizer:
    return self.optimizer(self.parameters(), **self.optimizer_params)
  
  def forward(self, x: torch.Tensor) -> torch.Tensor:
    return self.wrapped_model(x)
  
  def training_step(self, train_batch: Tuple[torch.Tensor, torch.Tensor], batch_idx: int) -> torch.Tensor:
    inputs, targets = train_batch
    outputs = self.wrapped_model(inputs)
    loss = self.loss_function(outputs, targets)
    self.log('train_loss', loss)
    return loss
  
  def validation_step(self, valid_batch: Tuple[torch.Tensor, torch.Tensor], batch_idx: int) -> torch.Tensor:
    inputs, targets = valid_batch
    outputs = self.wrapped_model(inputs)
    loss = self.loss_function(outputs, targets)
    self.log('valid_loss', loss)
    return loss
  
  def test_step(self, valid_batch: Tuple[torch.Tensor, torch.Tensor], batch_idx: int) -> torch.Tensor:
    inputs, targets = valid_batch
    outputs = self.wrapped_model(inputs)
    loss = self.loss_function(outputs, targets)
    self.log('test_loss', loss)
    return loss
  
  def predict_step(self, batch: torch.Tensor, batch_idx: int) -> torch.Tensor:
    inputs = batch
    outputs = self.wrapped_model(inputs)
    return torch.softmax(outputs)

Config Lightning Trainer

In [15]:
progress_bar = RichProgressBar(
    theme=RichProgressBarTheme(
      description="green_yellow",
      progress_bar="green1",
      progress_bar_finished="green1",
      progress_bar_pulse="#6206E0",
      batch_progress="green_yellow",
      time="grey82",
      processing_speed="grey82",
      metrics="grey82",
    )
)

model_checkpoint = ModelCheckpoint(
    dirpath='/content/gdrive/MyDrive/lightning_logs/finetuning_resnet50_fv/',
    save_last=True,
    every_n_epochs=1
)

# Training

In [ ]:
# Model and trainer
lightning_model = LightningWrapper(model) if do_training else LightningWrapper.load_from_checkpoint('/content/gdrive/MyDrive/lightning_logs/finetuning_resnet50_fv/last.ckpt')

device = 'gpu' if torch.cuda.is_available() else 'cpu'
trainer = pl.Trainer(accelerator=device, max_epochs=50, callbacks=[progress_bar, model_checkpoint])

# Dataloaders
train_loader = DataLoader(train_set, batch_size=512, num_workers=2)
valid_loader = DataLoader(valid_set, batch_size=512, num_workers=2)
test_loader = DataLoader(test_set, batch_size=512, num_workers=2)

if do_training:
  # Training
  trainer.fit(lightning_model, train_dataloaders=train_loader, val_dataloaders=valid_loader)

# Testing

In [ ]:
trainer.test(lightning_model, dataloaders=test_loader)

# Explicability / Interpretability with Captum

In [ ]:
img, tgt = train_set[0]
img = img.unsqueeze(0)
img.requires_grad = True

In [ ]:
deeplift = explain.DeepLift(lightning_model)
attribution = deeplift.attribute(img, target=tgt)

Le inplace est mauvais dans la construction de notre modèle (analysons le code) --> le inplace est utilisé pour la fonction d'activation (ReLU)